# Este script baja imagen segun un aoi dibujado interactivamente, y luego, clipea con el aoi (sobre la imagen ya bajada)
* busca imagenes por filtros
* genera un calculo de overlapping con el aoi determinado
* filtra de la base total aquellas imagenes que cumplan con un criterio (por ej, :x% overlaping)
* activa el asset que se quiera, y baja las imagenes zipeadas en un directorio a definir


In [1]:
import sys
import os
import json
import scipy
import urllib
import datetime 
import urllib3
import rasterio
import subprocess
import numpy as np
import pandas as pd
import seaborn as sns
from osgeo import gdal
from planet import api
from planet.api import filters
from traitlets import link
import rasterio.mask as rio_mask
from shapely.geometry import mapping, shape
from IPython.display import display, Image, HTML
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
urllib3.disable_warnings()
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

api_keys = json.load(open("apikeys.json",'r'))
client = api.ClientV1(api_key=api_keys["PLANET_API_KEY"])


# Make a slippy map to get GeoJSON
The planet API allows you to query using a geojson which is a special flavor of json.
We are going to create a slippy map using leaflet and apply the Planet 2017 Q1 mosaic as the basemap. This requires our api key.
We are going to add a special draw handler that shoves a draw region into a object so we get the geojson.
If you don't want to do this, or need a fixed query try geojson.io
To install and run:
$ pip install ipyleaflet
$ jupyter nbextension enable --py --sys-prefix ipyleaflet
$ jupyter nbextension enable --py --sys-prefix widgetsnbextension
More information

## filtros y definir aoi

In [2]:
AOI = json.load(open("estanzuela.geojson",'r'))
print(AOI['properties']['name'])
myAOI=AOI["geometry"]
print(myAOI)

# build a query using the AOI and
# a cloud_cover filter that excludes 'cloud free' scenes

oldthan = datetime.datetime(year=2017,month=10,day=20)
lessthan= datetime.datetime(year=2018,month=4,day=15)

query = filters.and_filter(
    filters.geom_filter(myAOI),
    filters.range_filter('cloud_cover', lt=5),
    filters.date_range('acquired', gt=oldthan,lte=lessthan)
)

# build a request for only PlanetScope imagery
request = filters.build_search_request(
    query, item_types=['PSScene4Band']
)

# if you don't have an API key configured, this will raise an exception
result = client.quick_search(request)
#print(result.items_iter)
scenes = []
planet_map = {}
for item in result.items_iter(limit=500):
    planet_map[item['id']]=item
    props = item['properties']
    props["id"] = item["id"]
    props["geometry"] = item["geometry"]
    props["thumbnail"] = item["_links"]["thumbnail"]
    #print(props)
    scenes.append(props)
    #print(scenes)
scenes = pd.DataFrame(data=scenes)
display(scenes)
print(len(scenes))

estanzuela
{'type': 'Polygon', 'coordinates': [[[-57.72538661956788, -34.35356956539674], [-57.718477249145515, -34.35356956539674], [-57.718477249145515, -34.347227415270744], [-57.72538661956788, -34.347227415270744], [-57.72538661956788, -34.35356956539674]]]}


,acquired,anomalous_pixels,cloud_cover,columns,epsg_code,geometry,ground_control,gsd,id,instrument,...,quality_category,rows,satellite_id,strip_id,sun_azimuth,sun_elevation,thumbnail,updated,usable_data,view_angle
0,2018-04-10T14:56:58.299594Z,0.00,0.03,8460,32721,"{'coordinates': [[[-57.74939759253474, -34.341...",True,3.7,20180410_145658_1053,PS2,...,standard,4217,1053,1341859,19.6,45.7,https://api.planet.com/data/v1/item-types/PSSc...,2018-04-10T18:00:31Z,0,0.2
1,2018-04-10T14:56:57.337554Z,0.00,0.03,8459,32721,"{'coordinates': [[[-57.732336957133576, -34.41...",True,3.7,20180410_145657_1053,PS2,...,standard,4215,1053,1341859,19.6,45.6,https://api.planet.com/data/v1/item-types/PSSc...,2018-04-10T18:00:31Z,0,0.3
2,2018-04-10T13:19:07.483235Z,0.00,0.39,9090,32721,"{'coordinates': [[[-57.93927162165936, -34.322...",True,4.0,20180410_131907_103d,PS2,...,test,4500,103d,1341765,47.9,34.3,https://api.planet.com/data/v1/item-types/PSSc...,2018-04-10T17:54:16Z,0,1.0
3,2018-04-10T13:19:06.439391Z,0.00,0.39,9085,32721,"{'coordinates': [[[-57.9173439454121, -34.2395...",True,4.0,20180410_131906_103d,PS2,...,test,4500,103d,1341765,48.0,34.3,https://api.planet.com/data/v1/item-types/PSSc...,2018-04-10T17:54:16Z,0,0.9
4,2018-04-09T13:19:02.834532Z,0.01,0.00,9067,32721,"{'coordinates': [[[-57.57449354618137, -34.402...",True,4.0,20180409_131902_1040,PS2,...,standard,4524,1040,1339306,48.3,34.5,https://api.planet.com/data/v1/item-types/PSSc...,2018-04-10T08:12:44Z,0,0.8
5,2018-04-09T14:57:24.758792Z,0.00,0.00,8357,32721,"{'coordinates': [[[-57.76427621010966, -34.351...",True,3.6,20180409_145724_0f46,PS2,...,standard,4151,0f46,1339426,19.7,46.0,https://api.planet.com/data/v1/item-types/PSSc...,2018-04-10T09:07:32Z,0,0.2
6,2018-04-05T13:18:56.738579Z,0.01,0.12,9069,32721,"{'coordinates': [[[-57.74174394598662, -34.393...",True,3.9,20180405_131856_0f52,PS2,...,standard,4517,0f52,1329716,49.6,35.5,https://api.planet.com/data/v1/item-types/PSSc...,2018-04-06T04:31:46Z,0,0.3
7,2018-04-05T13:18:55.593009Z,0.01,0.24,9055,32721,"{'coordinates': [[[-57.45302009086005, -34.397...",False,3.9,20180405_131855_0f52,PS2,...,test,4525,0f52,1329716,49.6,35.5,https://api.planet.com/data/v1/item-types/PSSc...,2018-04-06T04:31:39Z,0,0.3
8,2018-04-04T13:19:39.965819Z,0.00,0.06,9093,32721,"{'coordinates': [[[-57.94791516837868, -34.382...",True,4.0,20180404_131939_0f4e,PS2,...,standard,4511,0f4e,1327007,50.0,35.9,https://api.planet.com/data/v1/item-types/PSSc...,2018-04-05T05:28:31Z,0,1.6
9,2018-04-04T13:19:38.930656Z,0.00,0.01,9092,32721,"{'coordinates': [[[-57.65406567291055, -34.369...",True,4.0,20180404_131938_0f4e,PS2,...,standard,4513,0f4e,1327007,50.0,35.9,https://api.planet.com/data/v1/item-types/PSSc...,2018-04-05T05:24:36Z,0,1.6


257


# Cleanup
The data we got back is good, but we need some more information
We got back big scenes, but we only care about our area of interest. The scene may not cover the whole area of interest.
We can use the Shapely library to quickly figure out how much each scene overlaps our AOI
We will convert our AOI and the geometry of each scene to calculate overlap using a shapely call.
The returned acquisition, publish, and update times are strings, we'll convert them to datatime objects so we wan search.

In [3]:
# now let's clean up the datetime stuff
# make a shapely shape from our aoi
AOIs = shape(myAOI)
footprints = []
overlaps = []
# go through the geometry from our api call, convert to a shape and calculate overlap area.
# also save the shape for safe keeping
for footprint in scenes["geometry"].tolist():
    s = shape(footprint)
    footprints.append(s)
    overlap = 100.0*(AOIs.intersection(s).area / AOIs.area)
    overlaps.append(overlap)
# take our lists and add them back to our dataframe
scenes['overlap'] = pd.Series(overlaps, index=scenes.index)
scenes['footprint'] = pd.Series(footprints, index=scenes.index)
# now make sure pandas knows about our date/time columns.
scenes["acquired"] = pd.to_datetime(scenes["acquired"])
scenes["published"] = pd.to_datetime(scenes["published"])
scenes["updated"] = pd.to_datetime(scenes["updated"])
scenes.head()

,acquired,anomalous_pixels,cloud_cover,columns,epsg_code,geometry,ground_control,gsd,id,instrument,...,satellite_id,strip_id,sun_azimuth,sun_elevation,thumbnail,updated,usable_data,view_angle,overlap,footprint
0,2018-04-10 14:56:58.299594,0.00,0.03,8460,32721,"{'coordinates': [[[-57.74939759253474, -34.341...",True,3.7,20180410_145658_1053,PS2,...,1053,1341859,19.6,45.7,https://api.planet.com/data/v1/item-types/PSSc...,2018-04-10 18:00:31,0,0.2,20.539142,POLYGON ((-57.74939759253474 -34.3412439900209...
1,2018-04-10 14:56:57.337554,0.00,0.03,8459,32721,"{'coordinates': [[[-57.732336957133576, -34.41...",True,3.7,20180410_145657_1053,PS2,...,1053,1341859,19.6,45.6,https://api.planet.com/data/v1/item-types/PSSc...,2018-04-10 18:00:31,0,0.3,100.000000,POLYGON ((-57.73233695713358 -34.4112170088097...
2,2018-04-10 13:19:07.483235,0.00,0.39,9090,32721,"{'coordinates': [[[-57.93927162165936, -34.322...",True,4.0,20180410_131907_103d,PS2,...,103d,1341765,47.9,34.3,https://api.planet.com/data/v1/item-types/PSSc...,2018-04-10 17:54:16,0,1.0,100.000000,POLYGON ((-57.93927162165936 -34.3224540710060...
3,2018-04-10 13:19:06.439391,0.00,0.39,9085,32721,"{'coordinates': [[[-57.9173439454121, -34.2395...",True,4.0,20180410_131906_103d,PS2,...,103d,1341765,48.0,34.3,https://api.planet.com/data/v1/item-types/PSSc...,2018-04-10 17:54:16,0,0.9,71.729302,POLYGON ((-57.9173439454121 -34.23950606443221...
4,2018-04-09 13:19:02.834532,0.01,0.00,9067,32721,"{'coordinates': [[[-57.57449354618137, -34.402...",True,4.0,20180409_131902_1040,PS2,...,1040,1339306,48.3,34.5,https://api.planet.com/data/v1/item-types/PSSc...,2018-04-10 08:12:44,0,0.8,100.000000,POLYGON ((-57.57449354618137 -34.4024904672994...


# Filtering our search using pandas.
* Using our dataframe we will filter the scenes to just what we want.
* First we want scenes with less than 10% clouds.
* Second we want standard quality images. Test images may not be high quality.
* Third well only look for scenes since January.
* Finally we will create a new data frame with our queries and print the results.

In [4]:
# Now let's get it down to just good, recent, clear scenes
clear = scenes['cloud_cover']<0.1
good = scenes['quality_category']=="standard"

# definir que es recent, aca esta puesto con el mismo valor de oldthan, pero si el set es grande, hay que corregir.
recent = scenes["acquired"] > oldthan
#datetime.date(year=2018,month=3,day=1)

partial_coverage = scenes["overlap"] >90
good_scenes = scenes[(good&clear&recent&partial_coverage)]
good_scenes=good_scenes.sort_values(['acquired'], ascending=[False])
display(good_scenes)
print (len(good_scenes))

# Now let's get it down to just good, recent, clear scenes
clear = scenes['cloud_cover']<0.5
good = scenes['quality_category']=="standard"
all_time = scenes["acquired"] > datetime.date(year=2017,month=11,day=1)
full_coverage = scenes["overlap"] >= 60
all_scenes = scenes[(good&clear&all_time&full_coverage)]
#display(all_scenes)
print (len(all_scenes))

,acquired,anomalous_pixels,cloud_cover,columns,epsg_code,geometry,ground_control,gsd,id,instrument,...,satellite_id,strip_id,sun_azimuth,sun_elevation,thumbnail,updated,usable_data,view_angle,overlap,footprint
1,2018-04-10 14:56:57.337554,0.00,0.03,8459,32721,"{'coordinates': [[[-57.732336957133576, -34.41...",True,3.7,20180410_145657_1053,PS2,...,1053,1341859,19.6,45.6,https://api.planet.com/data/v1/item-types/PSSc...,2018-04-10 18:00:31,0,0.3,100.000000,POLYGON ((-57.73233695713358 -34.4112170088097...
5,2018-04-09 14:57:24.758792,0.00,0.00,8357,32721,"{'coordinates': [[[-57.76427621010966, -34.351...",True,3.6,20180409_145724_0f46,PS2,...,0f46,1339426,19.7,46.0,https://api.planet.com/data/v1/item-types/PSSc...,2018-04-10 09:07:32,0,0.2,100.000000,POLYGON ((-57.76427621010966 -34.3513138173902...
4,2018-04-09 13:19:02.834532,0.01,0.00,9067,32721,"{'coordinates': [[[-57.57449354618137, -34.402...",True,4.0,20180409_131902_1040,PS2,...,1040,1339306,48.3,34.5,https://api.planet.com/data/v1/item-types/PSSc...,2018-04-10 08:12:44,0,0.8,100.000000,POLYGON ((-57.57449354618137 -34.4024904672994...
8,2018-04-04 13:19:39.965819,0.00,0.06,9093,32721,"{'coordinates': [[[-57.94791516837868, -34.382...",True,4.0,20180404_131939_0f4e,PS2,...,0f4e,1327007,50.0,35.9,https://api.planet.com/data/v1/item-types/PSSc...,2018-04-05 05:28:31,0,1.6,98.953856,POLYGON ((-57.94791516837868 -34.3826616639164...
9,2018-04-04 13:19:38.930656,0.00,0.01,9092,32721,"{'coordinates': [[[-57.65406567291055, -34.369...",True,4.0,20180404_131938_0f4e,PS2,...,0f4e,1327007,50.0,35.9,https://api.planet.com/data/v1/item-types/PSSc...,2018-04-05 05:24:36,0,1.6,100.000000,POLYGON ((-57.65406567291055 -34.3691647576553...
16,2018-03-31 14:58:36.119286,0.00,0.00,8484,32721,"{'coordinates': [[[-57.47441510595113, -34.371...",True,3.7,20180331_145836_0f3b,PS2,...,0f3b,1316704,21.6,49.2,https://api.planet.com/data/v1/item-types/PSSc...,2018-04-01 05:34:27,0,0.1,100.000000,POLYGON ((-57.47441510595113 -34.3719425255552...
15,2018-03-31 13:19:01.060302,0.00,0.00,9065,32721,"{'coordinates': [[[-57.90661793475065, -34.335...",True,3.9,20180331_131901_1042,PS2,...,1042,1316449,51.6,36.7,https://api.planet.com/data/v1/item-types/PSSc...,2018-04-02 01:59:45,0,0.9,100.000000,POLYGON ((-57.90661793475065 -34.3354115800463...
18,2018-03-30 13:18:04.300262,0.01,0.00,9047,32721,"{'coordinates': [[[-57.851511452149175, -34.35...",True,3.9,20180330_131804_102f,PS2,...,102f,1314129,52.1,36.8,https://api.planet.com/data/v1/item-types/PSSc...,2018-03-31 05:35:10,0,0.2,100.000000,POLYGON ((-57.85151145214918 -34.3579719874956...
20,2018-03-29 13:18:26.519928,0.00,0.00,9064,32721,"{'coordinates': [[[-57.49967960087624, -34.412...",True,4.0,20180329_131826_0f3f,PS2,...,0f3f,1310647,52.4,37.2,https://api.planet.com/data/v1/item-types/PSSc...,2018-03-30 17:00:39,0,0.1,100.000000,POLYGON ((-57.49967960087624 -34.4122086413772...
24,2018-03-26 13:18:02.051295,0.00,0.00,9148,32721,"{'coordinates': [[[-57.7444432904199, -34.2920...",True,4.0,20180326_131802_103e,PS2,...,103e,1298424,53.6,37.9,https://api.planet.com/data/v1/item-types/PSSc...,2018-03-27 10:20:59,0,0.9,100.000000,POLYGON ((-57.7444432904199 -34.29200174948428...


128
152


## calcular el area total que ocupan las "good scenes"

In [5]:
#import pyproj
#p = pyproj.Proj(proj='utm', zone=21, ellps='WGS84')
#x,y = p(7.8509671, 47.9941214)
#print (x,y)
#414278.16731 5316285.59492
#print (p(x,y,inverse=True))
#(7.850967099999812, 47.994121399999784)

# Product Activation and Downloading
There are two things we need to know, the satellite type (asset) and image type (product).
Full resolution uncompressed satellite images are big and there are lots of ways to view them.
For this reason Planet generally keeps images in their native format and only processes them on customer requests. There is some caching of processed scenes, but this is the exception not the rule.
All images must be activated prior to downloading and this can take some time based on demand.
Additionally we need to determine what sort of product we want to download. Generally speaking there are three kinds of scenes:
Analytic - multi-band full resolution images that have not been processed. These are like raw files for DSLR camers.
Visual - these are color corrected rectified tifs. If you are just starting out this is your best call.
UDM - Usable data mask. This mask can be used to find bad pixels and columns and to mask out areas with clouds.

In [7]:
import requests
import time

# Set Item Type
item_type = 'PSScene4Band'

# Set Asset Type
asset_type = 'analytic_sr'

#lista de los nombres de imagenes a bajar (era scene_id)
to_get=good_scenes['id'].tolist()
#print(to_get)
##buscar la manera de que toget sea un dict
to_get=good_scenes['id']=1


api_key=api_keys["PLANET_API_KEY"]
url_base=[]
i=13
for scenes in to_get[13:18]:
# Request clip of scene (This will take some time to complete)
    #calcular el area de la seleccion de imagenes hecha e imprimir#
    try:
        clip_payload = {'aoi': myAOI,'targets': [{'item_id': to_get[i],'item_type': item_type,'asset_type': asset_type}]}
        #print(clip_payload)
        #
        request = requests.post('https://api.planet.com/compute/ops/clips/v1', auth=(api_key, ''), json=clip_payload)
        print(request)
        i=i+1
        clip_url= request.json()['_links']['_self']
        print(clip_url)
        # Poll API to monitor clip status. Once finished, download and upzip the scene
        clip_succeeded = False
        while not clip_succeeded:

        # Poll API
            check_state_request = requests.get(clip_url, auth=(api_key, ''))
            #print(check_state_request)
        # If clipping process succeeded , we are done
            if check_state_request.json()['state'] == 'succeeded':
                clip_download_url = check_state_request.json()['_links']['results'][0]
                clip_succeeded = True
                #print(clip_download_url)
                if clip_download_url in url_base: continue
                url_base.append(clip_download_url)
                print((i-1),"Clip of scene succeeded and is ready to download") 

        # Still activating. Wait 1 second and check again.
            else:
                print("...Still waiting for clipping to complete...")
                time.sleep(5)
        
    except:
        continue
print(len(url_base))

<Response [202]>
https://api.planet.com/compute/ops/clips/v1/115c6637-ed5b-4c8d-bea2-f0e04fddd901
...Still waiting for clipping to complete...
13 Clip of scene succeeded and is ready to download
<Response [401]>
<Response [401]>
<Response [401]>
<Response [202]>
https://api.planet.com/compute/ops/clips/v1/63e2757d-1d1a-4166-b2b3-82f0f7d765c0
...Still waiting for clipping to complete...
17 Clip of scene succeeded and is ready to download
2


Download and upzip the clip
Once complete, look in the output directory to see your clipped tif file.

NOTE: Clipped scene will only be available for 5 minutes.

In [20]:
import os
from tqdm import tqdm
import zipfile
from datetime import datetime

# ubicar los archivos donde se desee
path_or='D:/javie/Descargas/'

print(to_get[13:30])
#print(url_base)
i=13   
downloaded=[]
for url in url_base[0:20]: 
    try:
        response = requests.get(url, stream=True)
        #print(response)
        scene_id=to_get[i]
        print(scene_id)
        path =os.path.join(path_or+ scene_id +'_'+ asset_type +'_'+ AOI['properties']['name'])
        #print(path)
        i= i+1
        if os.path.isfile(path + '.zip' ):
            print ("skipping, we allready have scene: ", scene_id)
            continue 
        else:
            with open(path +'.zip', "wb") as handle:
                for data in tqdm(response.iter_content()):
                    #print(data)
                    handle.write(data)
            downloaded.append(path)
            print(downloaded)

            date=datetime.now()
            date=date.strftime('%Y_%m_%d_%H%M')

            newpath=os.path.join(path_or, date + '_downloaded.txt')
            #print(newpath)
            with open (newpath , "w") as pronto:
                for item in downloaded:
                    pronto.write("%s\n" % item)       
        #Unzip file
            ziped_item = zipfile.ZipFile(path + ".zip")
            ziped_item.extractall(path)     
        # Delete zip file
            time.sleep(5)
            os.remove(path + '.zip')
            print('Downloaded clips located in:', path_or)   
    except:
        continue
     


['20180322_131841_100a', '20180320_150053_0f46', '20180320_150052_0f46', '20180320_131923_0e2f', '20180319_131815_103c', '20180316_131852_0f3f', '20180314_150130_0f40', '20180314_131721_1044', '20180313_150108_101c', '20180313_131822_103e', '20180312_131736_0f18', '20180312_123747_1_0c59', '20180309_131739_101d', '20180308_131630_1018', '20180307_150238_0f4b', '20180306_131750_103c', '20180304_131617_1002']
20180322_131841_100a
skipping, we allready have scene:  20180322_131841_100a
20180320_150053_0f46


456222it [00:05, 86744.01it/s]


['D:/javie/Descargas/20180320_150053_0f46_analytic_sr_estanzuela']
